# 📈 Uplift Modeling Demo
This notebook demonstrates a basic two-model approach to estimate causal uplift using synthetic data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Simulate dataset
np.random.seed(42)
n = 1000
df = pd.DataFrame({
    'age': np.random.randint(18, 65, n),
    'income': np.random.normal(50000, 15000, n),
    'treatment': np.random.binomial(1, 0.5, n)
})
df['response'] = (0.3 * df['treatment'] + 
                  0.02 * (df['age'] > 30).astype(int) + 
                  0.01 * (df['income'] > 60000).astype(int) + 
                  np.random.normal(0, 0.1, n)) > 0.5
df['response'] = df['response'].astype(int)
df.head()

## 🎯 Build Separate Models for Treatment and Control

In [ ]:
# Split data
treat = df[df['treatment'] == 1]
control = df[df['treatment'] == 0]

# Train models
features = ['age', 'income']
model_t = LogisticRegression().fit(treat[features], treat['response'])
model_c = LogisticRegression().fit(control[features], control['response'])

# Predict uplift
df['uplift'] = model_t.predict_proba(df[features])[:,1] - model_c.predict_proba(df[features])[:,1]
df.sort_values('uplift', ascending=False).head(10)

## ✅ Use Case
Use uplift scores to target users most likely to respond **because of** a treatment—enabling more ethical and cost-efficient campaigns.